In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenantid/oauth2/token"}


dbutils.fs.mount(
    source = "abfss://landing-zone-2@ecomadlsthanos.dfs.core.windows.net",  #container@storageacc
    mount_point = "/mnt/ecomdata1",  # this is where adls while get mounted to databricks workspace
    extra_configs = configs
)

True

In [ ]:
%fs ls '/mnt/ecomdata1'

path,name,size,modificationTime
dbfs:/mnt/ecomdata1/buyers-raw-2/,buyers-raw-2/,0,1722861988000
dbfs:/mnt/ecomdata1/countries-raw-2/,countries-raw-2/,0,1722862006000
dbfs:/mnt/ecomdata1/sellers-raw-2/,sellers-raw-2/,0,1722861995000
dbfs:/mnt/ecomdata1/users-raw-2/,users-raw-2/,0,1722861982000


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.appName("EcomDataPipeline").getOrCreate()

In [ ]:
spark

In [ ]:
#read parquet file from /mnt/ecomdata1/user-raw-2 folder
usersDF = spark.read.format("parquet") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/mnt/ecomdata1/users-raw-2")

In [ ]:
# write this users data to the bronze delta lake table

usersDF.write.format("delta") \
    .mode("overwrite") \
    .save("/mnt/delta/tables/bronze/users")

In [ ]:
# create the DFs of the rest files (buyers,sellers,countries)
#write the DFs to the bronze layer delta tables

buyersDF = spark.read.format("parquet") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/mnt/ecomdata1/buyers-raw-2")

In [ ]:
buyersDF.write.format("delta") \
    .mode("overwrite") \
    .save("/mnt/delta/tables/bronze/buyers")

In [ ]:
sellersDF = spark.read.format("parquet") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/mnt/ecomdata1/sellers-raw-2")

In [ ]:
sellersDF.write.format("delta") \
    .mode("overwrite") \
    .save("/mnt/delta/tables/bronze/sellers")

In [ ]:
countriesDF = spark.read.format("parquet") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/mnt/ecomdata1/countries-raw-2")

In [ ]:
countriesDF.write.format("delta") \
    .mode("overwrite") \
    .save("/mnt/delta/tables/bronze/countries")